# Promedio Diario `PM2.5`

Para nuestra transformación de datos, utilizaremos las siguientes librerias:


In [1]:
import pandas as pd
import numpy as np

- Cargando dataset a un DataFrame de pandas
- Posteriormente normalizando la columna `Fecha` a tipo `datetime`

In [2]:
#Cargando archivos
df = pd.read_csv('../Datasets/PM2.5/PM25_depurado.csv')

# Normalizando fechas
df['FECHA'] = pd.to_datetime(df['FECHA'], unit='ns')
df.head()

,Unnamed: 0,FECHA,HORA,AJM,AJU,BJU,CAM,CCA,FAR,GAM,...,MON,NEZ,PED,SAC,SAG,SFE,TLA,UAX,UIZ,XAL
0,0,2019-01-01,1,19.0,35.0,62.0,90.0,66.0,NaN,NaN,...,102.0,133.0,NaN,NaN,92.0,33.0,55.0,74.0,99.0,96.0
1,1,2019-01-01,2,17.0,24.0,88.0,104.0,84.0,NaN,NaN,...,108.0,164.0,NaN,NaN,100.0,40.0,52.0,115.0,99.0,164.0
2,2,2019-01-01,3,14.0,20.0,107.0,140.0,95.0,NaN,NaN,...,125.0,206.0,NaN,NaN,104.0,52.0,59.0,150.0,109.0,249.0
3,3,2019-01-01,4,6.0,15.0,101.0,162.0,97.0,NaN,NaN,...,92.0,273.0,NaN,NaN,144.0,25.0,58.0,180.0,112.0,200.0
4,4,2019-01-01,5,4.0,8.0,121.0,133.0,88.0,NaN,NaN,...,84.0,291.0,NaN,NaN,171.0,21.0,46.0,167.0,164.0,161.0


- Reacomodo del DataFrame , donde ahora tendremos una columna para las estaciones y sus correspondientes datos de medición
- Eliminando los datos NaN
- Comprobando que no queden datos NaN en el DataFrame

In [3]:
# Extraer nombre de columnas
cols = df.columns.tolist()

# Reacomodando dataframe
df = pd.melt(df, id_vars=['FECHA', 'HORA'], value_vars=cols[3:], var_name='station', value_name='measurement')

# df = df[~df['measurement'].isna()]
df = df.dropna(axis=0, how='any').reset_index(drop=True)

print("No. de columnas que contienen valores nulos ")
print(len(df.columns[df.isna().any()]))

print("No. de columnas que no contienen valores nulos")
print(len(df.columns[df.notna().all()]))

print("No total de columnas en el dataframe")
print(len(df.columns))
df.head()

No. de columnas que contienen valores nulos 
0
No. de columnas que no contienen valores nulos
4
No total de columnas en el dataframe
4


,FECHA,HORA,station,measurement
0,2019-01-01,1,AJM,19.0
1,2019-01-01,2,AJM,17.0
2,2019-01-01,3,AJM,14.0
3,2019-01-01,4,AJM,6.0
4,2019-01-01,5,AJM,4.0


- Ordenando el dataframe segun la fecha y la estación

In [4]:
df = df.sort_values(['FECHA', 'station']).reset_index(drop=True)
df.head()

,FECHA,HORA,station,measurement
0,2019-01-01,1,AJM,19.0
1,2019-01-01,2,AJM,17.0
2,2019-01-01,3,AJM,14.0
3,2019-01-01,4,AJM,6.0
4,2019-01-01,5,AJM,4.0


### Calculando el promedio diario
Para el cálculo del promedio diario de PM10 y PM2.5 se utilizó la metodología descrita en la NOM-025-SSA1-2014, siguiendo la siguiente ecuación:

$$\bar{x}= \frac{1}{n} \displaystyle\sum_{i=1}^n x_i$$
$\bar{x}:$ promedio de 24 horas, 

$n:$ número de concentraciones horarias válidas

$x_i:$ concentraciones horarias válidas

- Camo datos extra obteniendo los valores minimos y maximos
- Tambien contabilizando cuantas horas fueron utilizadas para dicho promedio, columna(`N`)

In [5]:
# Calculando promedio diario, contando numero de horas usadas, obteniendo el maximo y minimo
df_PM2524h = df.groupby(['FECHA', 'station'], as_index=False).agg(PromDiario=('measurement', lambda x: round(x.mean(),0)), 
                                                                    N=('HORA', 'count'),
                                                                    Max = ('measurement', 'max'),
                                                                    Min = ('measurement', 'min'))
df_PM2524h.head()

,FECHA,station,PromDiario,N,Max,Min
0,2019-01-01,AJM,25.0,24,76.0,4.0
1,2019-01-01,AJU,15.0,22,35.0,3.0
2,2019-01-01,BJU,66.0,24,121.0,8.0
3,2019-01-01,CAM,68.0,23,162.0,13.0
4,2019-01-01,CCA,49.0,24,97.0,13.0


### Calculando Índice de calidad del aire

Donde:
$$\text{Índice}= (k \times (C \times BP_{Lo}))+I_{Lo}$$

$$k= \frac{I_{Hi}-I_{Lo}}{BP_{Hi}-BP_{Lo}}$$

 
$\text{Índice}:$ Valor del índice para el contaminante deseado.<br>
$C:$ valor redondeado para la concentración del contaminante.<br>
$k:$ constante de proporcionalidad estimada.<br>
$BP_{Hi}:$ valor del punto de corte que es mayor o igual a la concentración a evaluar.<br>
$BP_{Lo}:$ valor del punto de corte que es menor o igual a la concentración a evaluar.<br>
$I_{Hi}:$ valor del índice que corresponde al valor de $BP_{Hi}$<br>
$I_{Lo}:$ valor del índice que corresponde al valor de $BP_{Lo}$<br>

In [6]:
def AQI(df):
    CP = df['PromDiario']
    i = 0

    if 0.0 <= CP <= 12.0:
        k = (50-0)/(12-0)
        i = k * (CP - 0) + 0
    if 12.1 <= CP <= 45.0:
        k = (100-51)/(45-12.1)
        i = k * (CP - 12.1) + 51
    if 45.1 <= CP <= 97.4:
        k = (150-101)/(97.4-45.1)
        i = k * (CP - 45.1) + 101
    if 97.5 <= CP <= 150.4:
        k = (200-151)/(150.4-97.5)
        i = k * (CP - 97.5) + 151
    if 150.5 <= CP <= 250.4:
        k = (300-201)/(250.4-150.5)
        i = k * (CP - 150.5) + 201
    if 250.5 <= CP <= 350.4:
        k = (400-301)/(350.4-250.5)
        i = k * (CP - 250.5) + 301
    if 350.5 <= CP <= 500.4:
        k = (500-401)/(500.4-350.5)
        i = k * (CP - 350.5) + 401

    return round(i, 3)

In [7]:
df_PM2524h['indice'] = df_PM2524h.apply(AQI, axis=1)
df_PM2524h.head()

,FECHA,station,PromDiario,N,Max,Min,indice
0,2019-01-01,AJM,25.0,24,76.0,4.0,70.213
1,2019-01-01,AJU,15.0,22,35.0,3.0,55.319
2,2019-01-01,BJU,66.0,24,121.0,8.0,120.581
3,2019-01-01,CAM,68.0,23,162.0,13.0,122.455
4,2019-01-01,CCA,49.0,24,97.0,13.0,104.654


- Dejando los datos que utilicen mas de 17 horas para calcular el promedio diario
- Eliminando la columna N

In [8]:
df_PM2524h = df_PM2524h.query('N > 17')
df_PM2524h = df_PM2524h.drop(columns=['N'])
df_PM2524h.head()

,FECHA,station,PromDiario,Max,Min,indice
0,2019-01-01,AJM,25.0,76.0,4.0,70.213
1,2019-01-01,AJU,15.0,35.0,3.0,55.319
2,2019-01-01,BJU,66.0,121.0,8.0,120.581
3,2019-01-01,CAM,68.0,162.0,13.0,122.455
4,2019-01-01,CCA,49.0,97.0,13.0,104.654


- Cargando dataset de las estaciones con su correspondiente zona

In [9]:
df_zonas = pd.read_csv('../Datasets/cat_estacion_depurado.csv')

# Eliminando columna 'Unnamed: 0'
df_zonas = df_zonas.drop(columns=['Unnamed: 0'])
# Remplezando los - por NaN
df_zonas = df_zonas.replace('-', np.nan)
# Eliminando NaN's
df_zonas = df_zonas.dropna(axis=0, how='any').reset_index(drop=True)
# Cambiando de nombre la columna 'cve_estac'
df_zonas = df_zonas.rename(columns={'cve_estac': 'station'})
df_zonas.head()

,station,nom_estac,Zona
0,ACO,Acolman,NE
1,AJU,Ajusco,SO
2,AJM,Ajusco Medio,SO
3,ATI,Atizapan,NO
4,BJU,Benito Ju�rez,CE


- Uniendo los datos con las zonas a cuales les corresponde

In [10]:
df_PM2524h = df_PM2524h.merge(df_zonas[['station', 'Zona']], on='station')
df_PM2524h.head()

,FECHA,station,PromDiario,Max,Min,indice,Zona
0,2019-01-01,AJM,25.0,76.0,4.0,70.213,SO
1,2019-01-02,AJM,14.0,33.0,3.0,53.830,SO
2,2019-01-03,AJM,23.0,32.0,9.0,67.234,SO
3,2019-01-04,AJM,22.0,31.0,14.0,65.745,SO
4,2019-01-05,AJM,18.0,39.0,5.0,59.787,SO


- Separando la fecha por, año, mes y dia

In [11]:
# Creando dataframe temporal para separar la fecha en años, meses y dias
dfdate = pd.DataFrame(columns = ['Year', 'Month', 'Day'])
# Extrayendo años
dfdate['Year'] = df_PM2524h['FECHA'].dt.year
# Extrayendo meses
dfdate['Month'] = df_PM2524h['FECHA'].dt.month
# Extrayendo dias
dfdate['Day'] = df_PM2524h['FECHA'].dt.day

# Elimiando columna de fecha en el dataframe original
df_PM2524h = df_PM2524h.drop(columns=['FECHA'])

# concatenacion del dataframe temporal que contiene la fecha separadas
data_PM2524h = pd.concat([dfdate.reset_index(drop=True), df_PM2524h.reset_index(drop=True)], axis=1)
data_PM2524h.head()

,Year,Month,Day,station,PromDiario,Max,Min,indice,Zona
0,2019,1,1,AJM,25.0,76.0,4.0,70.213,SO
1,2019,1,2,AJM,14.0,33.0,3.0,53.830,SO
2,2019,1,3,AJM,23.0,32.0,9.0,67.234,SO
3,2019,1,4,AJM,22.0,31.0,14.0,65.745,SO
4,2019,1,5,AJM,18.0,39.0,5.0,59.787,SO


- Guardando dataframe en un csv

In [12]:
data_PM2524h.to_csv('../Datasets/PM2.5/PDEstacion_PM25.csv', index=False)

### Calculo de promedio mensual

In [13]:
zona_ave = data_PM2524h.groupby(['Year','Month','Zona'], as_index=False).agg(
                                                                PromMensual=('PromDiario', lambda x: round(x.mean(),1)))
zona_ave.head()

,Year,Month,Zona,PromMensual
0,2019,1,CE,23.3
1,2019,1,NE,25.4
2,2019,1,NO,24.5
3,2019,1,SE,23.3
4,2019,1,SO,16.9


- Reordenando el dataframe en donde cada zona tenga una columa con sus valores correspondientes

In [14]:
zona_ave25 = pd.pivot_table(zona_ave, index=[zona_ave.Year, zona_ave.Month], columns='Zona', values='PromMensual').reset_index()

zona_ave25.Year = zona_ave.Year.astype(int)

# zona_ave.dtypes
zona_ave25.head()

Zona,Year,Month,CE,NE,NO,SE,SO
0,2019,1,23.3,25.4,24.5,23.3,16.9
1,2019,2,20.9,22.7,22.7,21.0,15.5
2,2019,3,22.2,23.9,24.7,22.8,18.4
3,2019,4,28.6,30.9,33.3,29.9,26.3
4,2019,5,31.4,33.1,33.0,33.6,31.1


- Guardando resultados en un csv

In [15]:
zona_ave25.to_csv('../Datasets/PM2.5/PMZona_PM25.csv')